![figure](lab3/politecnico_h-01.png)

# **Eletrónica Programável**
#### Licenciatura em Engenharia Eletrotécnica e de Computadores

## Trabalho Laboratorial 3 - Circuitos Sequenciais 

-----------------------


## Introdução ##
Neste trabalho vamos implementar um conjunto de circuitos eletrónicos sequenciais usando uma descrição comportamental em VHDL, que permite implementar circuitos de média escala usando um nível de abstração superior. Este documento não fornece detalhes de utilização do Vivado. Sempre que tiver dúvidas a este nível, consulte o enunciado do Lab1.


### Objetivos ###

Neste trabalho pretende-se observar como a especificação comportamental pode ser usada para implementar diferentes tipos de circuitos sequenciais de média escala, como registos, memórias, contadores. Para tal, propomos para isso a implementação de 3 circuitos sequenciais :
* Sistema de multiplexagem temporal para visores de 7 segmentos.
* Registo FIFO (Fist-In-First-Out).
* Cronómetro digital.


Nas instruções abaixo **{sources}** refere-se a `C:\Xilinx\EEC_EP\sources` e **{labs}** refere-se a `(C:\Xilinx\EEC_EP\labs)`


### Referencias ###

Pong P. Chu. 2008. FPGA Prototyping by VHDL Examples: Xilinx Spartan-3 Version. 


_______________________

## Passo 1 - Sistema de multiplexagem temporal para visores de 7 segmentos ##

### Passo 1.1 ### 

O interface a visores de 7 segmentos com vários elementos é geralmente realizado por varrimento, para reduzir a utilização de pinos da FPGA. Isto significa que as entradas do conjunto de visores são multiplexadas no tempo. Cada visor possui uma entrada de habilitação individual ($an_i$, i=1...4) mas partilha as entradas de dados com todos os outros visores. Note que a entrada de habilitação é asserida a zero, como mostra a figura. Desde que a taxa de refrescamento seja suficientemente elevada, o olho humano não consegue perceber que os visores são alimentados por varrimento.


<div>
<img src="lab3/fig1.png" width="800"/>
</div>

A multiplexagem temporal pode ser realizada antes ou depois da conversão de hexadecimal para 7 segmentos (veja a figura abaixo). Uma vez que a representação binária é mais comum na maioria das aplicações, vamos considerar que a conversão para 7 segmentos é realizada depois do circuito de multiplexagem. Este esquema requer um descodificador de hexadecimal para 7 segmentos, um multiplexador de vectores de 4 bits, um contador e um descodificador 2:4 asserido a zero. O código VHDL correspondente ao circuito representado na figura é fornecido no ficheiro **disp_hex_mux.vhd**.


<div>
<img src="lab3/fig2.png" width="800"/>
</div>


A taxa de refrescamento deve ser superior a 100Hz para garantir que o varrimento não é perceptível ao olho humano. A placa de desenvolvimento vem equipada com um oscilador de $f_{osc}$ =100MHz, pelo que precisamos de um divisor de relógio. O circuito mais simples para implementar um divisor de relógio é um contador. O LSB (bit 0) de um contador oscila à frequência $f_{osc}$ / $2$. Por analogia, o bit n irá oscilar a uma frequência $f_{osc}$ / $2^{n+1}$. Logo, o bit n=17 de um contador binário oscila à frequência 100MHz / $2^{18}$ = 381Hz.


* Comece por descarregar os arquivos fonte fornecidos (Moodle) e salve-os em `{sources}/lab3`.


* Abra o Vivado a partir do icon no ambinete de trabalho ou: `Start > All Programs > Xilinx Design Tools > Vivado 2022.1`. 


* Selecione **Create Project** para começar o wizard. Na caixa de diálogo *Create A New Vivado Project*, selecione **Next**.


* Clique no botão *Procurar* do campo de localização do projeto do novo formulário de projeto, navegue até `c:/Xilinx/EEC_EP/labs` e clique em **Select**. Digite *lab3* no campo de nome do projeto. Certifique-se de que a caixa *Create Project Subdirectory* esteja marcada. Clique em **Next**.


* Vamos criar um projeto RTL vazio. Especificaremos as fontes posteriormente. Selecione a opção **RTL Project** no formulário *Project Type* e clique em **Next**. Certifique-se de que a caixa *Do not specify sources at this time* está marcada.


* No formulário **Default Part**, selecione a opção *Boards* e escolha a placa **Basys3**. Clique em **Next** e depois em **Finish** para criar o projeto Vivado.


* No painel *Sources*, selecione **Add Sources** e siga os passos necessários (*Add or create design sources*) para adicionar o circuito somador fornecido no ficheiro **disp_hex_mux.vhd**. 


* Abra o ficheiro no Vivado, observe o código. Verifique que o código implementa de facto o diagrama de blocos explicado acima.



### Passo 1.2 ### 

Nesta seção, vamos verificar o funcionamento do circuito em simulação. 

O facto de usarmos um contador de 18 bits para gerar um sinal de relógio de baixa frequência, torna a simulação deste circuito demasiadamente longa - cada iteração demora 218 ciclos de relógio (o que não faz sentido se queremos apenas verificar o comportamento). A solução passa por reduzir o tamanho do contador, alterando o valor da constante **N**. 

* Abra o ficheiro fonte (**disp_hex_mux.vhd**) e altere a linha onde a constante **N** é definida para: `constant N: integer :=4;` e faça **SAVE**.


* Adicione agora o ficheiro de estímulos (*Add or create simulation sources*) fornecido **disp_hex_mux_tb.vhd**. Deve ficar com a seguinte hierarquia de ficheiros na janela *Sources*.

<div>
<img src="lab3/fig3.png" width="800"/>
</div>

* Corra o processo **Run Simulation** e observe o resultado de simulação. Adicione à janela de ondas os sinais internos do UUT: **qreg** e **sel** e **hex**. Selecione também o **Radix** adequado para cada sinal. Deve obter algo como se mostra na figura abaixo. 


<div>
<img src="lab3/fig4.png" width="1000"/>
</div>


* Feche a vista de simulação na barra azul.


* Para evitar alterações de código entre a simulação e a síntese, altere o ficheiro **disp_hex_mux** de forma a que a constante **N seja definida como generic**. Assim, bastará alterar o seu valor quando o módulo for utilizado em síntese (*module instantiation*), que é o caso do próximo passo.

### Passo 1.3 ### 

Nesta seção, vamos verificar o funcionamento do circuito na placa, considerando que 8 interruptores definem dois números hexadecimais, que devem ser visualizados em dois visores de 7 segmentos. Os outros dois visores de 7 segmentos devem mostrar o resultado da soma dos dois números Hexadecimais.


* No painel *Sources*, selecione **Add Sources** e siga os passos necessários para adicionar o ficheiro: **hex_mux_test.vhd**. Abra o ficheiro no Vivado e observe o código.


* Adicione a linha que mapeia a contante genérica **N** no valor **18**: `generic map (N => 18)`


* Escolha a operação *Open Elaborated Design* e abra o esquemático correspondente a este circuito. Deve ter algo como se mostra na figura.


<div>
<img src="lab3/fig5.png" width="1000"/>
</div>


* Feche a vista *Elaborated Design* na barra azul.


* Adicione o ficheiro (*Constraints file*) fornecido **basys3_master.xdc**. Faça as alterações necessárias para atribuir as entradas e saídas do circuito aos pinos adequados da placa **Basys3**. Faça **Save**.


* Lembre-se de que o Pino W7 corresponde ao segmento 6 (sseg[6]) e por essa ordem, o pino U7 corresponde ao segmento 0 (sseg[0]). O ponto do visor (pino V7) corresponde ao segmento 7 (sseg[7]).


<div>
<img src="lab3/fig5a.png" width="600"/>
</div>


* Só falta agora correr os processos de síntese, implementação e geração de *bitstream*. Selecione o ficheiro **hex_mux_test.vhd**, faça duplo clique em **Generate Bitstream** e todos os processos necessários serão efectuados.


* Ligue a placa ao PC usando o cabo USB e programe a placa. Verifique o funcionamento do circuito. 


* Feche a vista *Hardware Manager* na faixa azul e desligue a Basys3 no interruptor da placa.

______

## Passo 2 - Registo FIFO (Fist-In-First-Out). ##

### Passo 2.1 ### 

Pretende-se implementar um registo elástico FIFO que permita fazer o interface entre dois subsistemas, como exemplifica o diagrama conceptual da figura. 

<div>
<img src="lab3/fig6.png" width="300"/>
</div>


Este tipo de circuito é fundamental para permitir a comunicação entre dois subsistemas assíncronos, isto é, que não partilhem do mesmo sinal de relógio (como os subsistemas de transmissão e receção, num sistema de comunicação digital). 
O circuito tem dois sinais de controlo, **wr** e **rd**, para controlar a escrita (wr) e a leitura (rd) de dados. 


Na realidade não existe de facto uma leitura – os dados na ponta do FIFO estão sempre disponíveis para leitura. O sinal **rd** apenas remove os dados da ponta e avança o ponteiro de leitura para a posição seguinte, que passa a ser a ponta. 


Uma forma de implementar um FIFO é adicionando um circuito de controlo a um registo circular com dois ponteiros. O ponteiro de escrita aponta para a cabeça do registo enquanto o ponteiro de leitura aponta para a cauda. A cada operação de escrita/leitura o ponteiro correspondente avança uma posição. 


Na figura abaixo é representado o funcionamento de um FIFO de 8 bits, com pedidos de escrita e leitura. Geralmente este tipo de registos incluem também indicadores de memória cheia (full) e memória vazia (empty), que ocorrem quando os ponteiros de escrita e leitura estão na mesma posição. Para conseguir distinguir as duas situações são utilizados flip-flops, um inicializado a ‘0’ e outro a ‘1’. De acordo com as operações de escrita e leitura, o valor dos flip-flops vai sendo atualizado de forma a que seja sempre possível distinguir os estados full e empty. 



<div>
<img src="lab3/fig7.png" width="600"/>
</div>



* A descrição VHDL dum FIFO de 16 posições (4 bits de endereço) e palavras de 8 bits, é fornecida no ficheiro **fifo.vhd** Adicione-o ao projeto Vivado (*Add or create design sources*), abra o ficheiro e observe o código.


* Para verificar o funcionamento do circuito na placa vamos utilizar uma versão reduzida do FIFO, com dados de 3 bits (4-by-3 FIFO). Adicione o ficheiro **fifo_test.vhd**. Note que os dados de entrada são fornecidos por 3 interruptores, enquanto os comandos **rd** e **wr** são gerados através de botões de pressão. Os dados lidos, bem como os indicadores full e empty são apresentados em 8 LEDs. 


* Uma vez que os botões de pressão podem apresentar trepidações (bounce), são utilizados circuitos de debounce. A descrição VHDL é fornecida no ficheiro **debounce.vhd** (deixamos a análise detalhada deste circuito para o próximo trabalho). Os circuitos de debounce garantem que é gerado apenas um pulso cada vez que o utilizador pressiona o botão. Adicione este ficheiro ao projeto.


* Para  sintetizar e implementar o módulo **fifo_test.vhd**, deve primeiro pressionar o botão direito do rato sobre esse módulo (na janela *Sources*), e depois escolher a opção **Set as Top** que aparece na janela de diálogo. Esta opção permite definir o módulo como sendo o circuito a implementar. 


* O ficheiro de pinos (XDC) deve ficar automaticamente associado a este novo ficheiro. Altere então o ficheiro **basys3_master.xdc** para ficar de acordo com os requisitos deste circuito. 


* Só falta agora correr os processos de síntese, implementação e geração de *bitstream*. Faça duplo clique em **Generate Bitstream** e todos os processos necessários serão efectuados.


* Ligue a placa ao PC usando o cabo USB e programe a placa. Verifique o funcionamento do circuito. Escreva na FIFO os números **1**, **2**, **3** e **4**. Note que a FIFO fica cheia neste momento(LED_FULL = ON). Leia agora pela msma ordem (incemente a posição do ponteiro de leitura) e note que vai ver estes valores nos leds até a FIFO ficar vazia (LED_EMPTY = ON).


* Feche a vista *Hardware Manager* na faixa azul e desligue a Basys3 no interruptor da placa.


______________

## Passo 3 - 3.	Cronómetro digital ##

### Passo 3.1 ### 

Vamos agora implementar um cronómetro digital. O tempo é visualizado em 3 dígitos decimais, contando entre 00.0 e 99.9 segundos. Quando chega a 99.9 deve recomeçar automaticamente. Deve conter um clear síncrono para recomeçar a contagem, e um sinal de **go** para suspender/iniciar a contagem. O circuito baseia-se num contador BCD (Binary Coded Decimal), como mostra a figura abaixo. Neste formato, um número é representado por uma sequência de dígitos BCD de 4 bits. Por exemplo, o número $139_{10}$ é representado como “0001 0011 1001”. 



<div>
<img src="lab3/fig8.png" width="1000"/>
</div>



Uma vez que a placa Basys3 possui um oscilador de 100MHz, é necessário também implementar um contador Mod-10000000 para gerar um sinal de relógio com um período de 100 milissegundos (resolução temporal do cronómetro). Existem várias formas de implementar este circuito. No ficheiro **stop_watch.vhd**, é fornecida uma implementação baseada em 3 contadores BCD (contadores mod-10) em cascata e um contador Mod-10000000 para gerar o sinal de relógio de 1 milissegundo (ms_tick). Os contadores são todos controlados pelo sinal de relógio de entrada, embora tenham uma entrada de enable que permite definir quando é que estão habilitados a incrementar.


O contador das décimas de segundo (**d0**) é incrementado com o sinal de relógio de 10Hz (`enable<=’1’ when ms_tick=’1’ else ‘0’`). O contador dos segundos é incrementado sempre que o contador de décimas de segundo chega ao fim de contagem (**d0_reg=9**). Assim, este contador só avança uma unidade quando o anterior chegar ao fim. Da mesma forma, o contador de dezenas de segundos só é habilitado com o fim de contagem dos contadores anteriores (**d0_reg=9 and d1_reg=9**).


Para verificar o funcionamento deste circuito na placa de desenvolvimento da Digilent, vamos combinar este circuito com o código que permite usar o conjunto de 4 displays de 7 segmentos. O código (estrutural) do circuito de teste é fornecido no ficheiro **stop_watch_test.vhd**. Note que o primeiro dígito do visor (hex3) é colocado a zero e que os sinais **clr** e **go** são mapeados em botões de pressão. 


* Adicione estes dois ficheiros ao seu projeto e Analise o código de cada um dos ficheiros. 


* Selecione o ficheiro **stop_watch_test.vhd** e garanta que é o ficheiro de topo na janela **Sources**.


* Abra o ficheiro e verifique se o genérico N está bem mapeado: `generic map (N => 18)`


* Defina os pinos do circuito no ficheiro XDC.


* Siga os procedimentos necessários para sintetizar e implementar o circuito na placa da Digilent. 



_________

## Desafio ##


#### Desafio 1 ####
Simule o funcionamento do cronómetro digital da secção 1.3. Note que deve reduzir a constante DVSR para um valor mais baixo para evitar um tempo de simulação exagerado. Para tal, pode criar um genérico para esta constante e fazer o mapeamento ao nível do ficheiro de topo.

#### Desafio 2 ####

Implemente um circuito com uma FIFO de 8 posições, com possibilidade de visualizar o valor de saída (valor lido) num dos visores de 7 segmentos. A indicação de FIFO full e empty devem também poder ser visualizados num visor de 7 sgmentos:
* FIFO full -> segmento **a** aceso.
* FIFO empty -> segmento **d** aceso.
